# Pastesian Formulation
The statement of the use case is on Mip Wise's website: 
[mipwise.com/use-cases/pastesian](https://www.mipwise.com/use-cases/pastesian).

### Imports

In [11]:
import pulp

import warnings
warnings.filterwarnings("ignore")

## Formulation

### Decision Variables
The image below illustrates the flow of lasagnas through
time.

![Pastesian Flow](https://raw.githubusercontent.com/mipwise/use-cases/d7bcb1223aa74a4c30db440ebfda976912577bb8/pastesian/docs/pastesian_flow.png)


In [12]:
def name_constraint(model: pulp.LpProblem):
    for i in range(1, len(model.constraints.keys())+2):
        name = f'c{i}'
        if name not in model.constraints.keys():
            return name
        

In [13]:
pcosts = {1: 5.50,
          2: 7.20,
          3: 8.80,
          4: 10.90,
          }

scosts = {0: 0,
          1: 1.30,
          2: 1.95,
          3: 2.20,
          4: 0,
          }

demand = {1: 200,
          2: 350,
          3: 150,
          4: 250,
          }


In particular, this chart suggests that the decision variables we need to define are the amounts to be produced
in each month, and the amounts of inventory to carry over from one month to the next:
- $x_1$	- Number of lasagnas to be produced in month 1.
- $x_2$	- Number of lasagnas to be produced in month 2.
- $x_3$	- Number of lasagnas to be produced in month 3.
- $x_4$	- Number of lasagnas to be produced in month 4.
- $s_0$	- Initially stored 50 lasagnas.
- $s_1$	- Number of lasagnas stored from month 1 to month 2.
- $s_2$	- Number of lasagnas stored from month 2 to month 3.
- $s_3$	- Number of lasagnas stored from month 3 to month 4.
- $s_4$	- There will be no stored lasagnas after month 4.

In [14]:
p_keys = pcosts.keys()
s_keys = scosts.keys()

# Define the model
mdl = pulp.LpProblem('Minimize Costs', sense=pulp.LpMinimize)

# Add variables
p = pulp.LpVariable.dicts(indices=p_keys, cat=pulp.LpInteger, lowBound=0, name='p')
s = pulp.LpVariable.dicts(indices=s_keys, cat=pulp.LpInteger, lowBound=0, name='s')

s[0].setInitialValue(50)
s[0].fixValue()

s[4].setInitialValue(0)
s[4].fixValue()

### Constraints
* Flow balance constraint for month 1:
$$50 + x_1 = 200 + s_1.$$

* Flow balance constraint for month 2:
$$s_1 + x_2 = 350 + s_2.$$

* Flow balance constraint for month 3:
$$s_2 + x_3 = 150 + s_3.$$

* Flow balance constraint for month 4:
$$s_3 + x_4 = 250.$$

In [15]:
# Add Constraints
for i in p_keys:
  mdl.addConstraint(s[i-1] + p[i] == demand[i] + s[i], name=name_constraint(mdl))

### Objective
The objective is to minimize the total production and inventory cost.
$$\min{5.50 x_1 + 7.20 x_2 + 8.80 x_3 + 10.90 x_4 + 1.30 s_1 + 1.95 s_2 + 2.20 s_3}.$$

In [16]:
# Set the objective function
mdl.setObjective(pulp.lpSum(pcosts[i] * p[i] for i in p_keys)+pulp.lpSum(scosts[i] * s[i] for i in s_keys))

### Final formulation
$$
\begin{darray}{rcl}
\begin{array}{rcl}
& \min & 5.50 x_1 + 7.20 x_2 + 8.80 x_3 + 10.90 x_4 + 1.30 s_1 + 1.95 s_2 + 2.20 s_3\\
& \text{s.t.}& 50 + x_1 = 200 + s_1,\\
&& s_1 + x_2 = 350 + s_2,\\
&& s_2 + x_3 = 150 + s_3,\\
&& s_3 + x_4 = 250,\\
&& x_1, x_2, x_3, x_4, s_1, s_2, s_3 \geq 0.
\end{array}
\end{darray}
$$

In [17]:
mdl.solve()

# Optimize

p_sol = {i: p[i].value() for i in p_keys}
s_sol = {i: s[i].value() for i in s_keys}

print(f"Production Flow = {p_sol}\nStorage Flow = {s_sol}")
#print(f'Solução: {pulp.lpSum(costs[i] * x_sol[i] for i in x_keys)}')

Production Flow = {1: 650.0, 2: 0.0, 3: 0.0, 4: 250.0}
Storage Flow = {0: 50.0, 1: 500.0, 2: 150.0, 3: 0.0, 4: 0.0}


## Additional complexities

### New decision variables
- $y_t$	- Number of lasagnas to be produced beyond the regular capacity in month t.

In [18]:
y_keys = demand.keys()

y = pulp.LpVariable.dicts(indices=y_keys, cat=pulp.LpInteger, lowBound=0, name='y')

### New constraints
* Storage capacity:
$$
s_t \leq 200, \quad t=1, 2, 3, 4.
$$
* Procution capacity:
$$
x_t \leq 400, \quad t=1, 2, 3, 4.
$$
* Number of lasagnas produced beyond the regular capacity in month $t=1, 2, 3, 4$:
$$
x_t - 300 \leq y_t.
$$


In [19]:
for i in s_keys:
    mdl.addConstraint(s[i] <= 200, name=name_constraint(mdl))

In [22]:
for i in p_keys:
    mdl.addConstraint(p[i] <= 400, name=name_constraint(mdl))

In [25]:
for i in y_keys:
    mdl.addConstraint(p[i] - y[i] <= 300, name=name_constraint(mdl))

In [26]:
mdl

Minimize_Costs:
MINIMIZE
5.5*p_1 + 7.2*p_2 + 8.8*p_3 + 10.9*p_4 + 1.3*s_1 + 1.95*s_2 + 2.2*s_3 + 0.0
SUBJECT TO
c1: p_1 + s_0 - s_1 = 200

c2: p_2 + s_1 - s_2 = 350

c3: p_3 + s_2 - s_3 = 150

c4: p_4 + s_3 - s_4 = 250

c5: s_0 <= 200

c6: s_1 <= 200

c7: s_2 <= 200

c8: s_3 <= 200

c9: s_4 <= 200

c10: p_1 <= 400

c11: p_2 <= 400

c12: p_3 <= 400

c13: p_4 <= 400

c14: p_1 - y_1 <= 300

c15: p_2 - y_2 <= 300

c16: p_3 - y_3 <= 300

c17: p_4 - y_4 <= 300

VARIABLES
0 <= p_1 Integer
0 <= p_2 Integer
0 <= p_3 Integer
0 <= p_4 Integer
s_0 = 50 Integer
0 <= s_1 Integer
0 <= s_2 Integer
0 <= s_3 Integer
s_4 = 0 Integer
0 <= y_1 Integer
0 <= y_2 Integer
0 <= y_3 Integer
0 <= y_4 Integer



### Upadated objective function
$$\min{5.50 x_1 + 7.20 x_2 + 8.80 x_3 + 10.90 x_4 + 1.30 s_1 + 1.95 s_2 + 2.20 s_3 + 0.35(y_1 + y_2, + y_3 + y_4)}.$$

In [28]:
ycost = 0.35

# Set the objective function
mdl.setObjective(pulp.lpSum(pcosts[i] * p[i] for i in p_keys)
               + pulp.lpSum(scosts[i] * s[i] for i in s_keys)
               + pulp.lpSum(ycost * y[i] for i in y_keys))

In [30]:
mdl.solve()

# Optimize

p_sol = {i: p[i].value() for i in p_keys}
s_sol = {i: s[i].value() for i in s_keys}

print(f"Production Flow = {p_sol}\nStorage Flow = {s_sol}")
#print(f'Solução: {pulp.lpSum(costs[i] * x_sol[i] for i in x_keys)}')

Production Flow = {1: 350.0, 2: 150.0, 3: 150.0, 4: 250.0}
Storage Flow = {0: 50.0, 1: 200.0, 2: 0.0, 3: 0.0, 4: 0.0}
